In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import scipy.stats
import geopandas as gpd
%matplotlib inline
from IPython.display import Markdown
from functools import reduce
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go

In [3]:
from preprocessing import rename_columns, replace_with_nan
table_40 = pd.read_csv('data/RV_O_040_R_KR_SK.CSV', sep=';')
table_40['Vek'] = pd.to_numeric(table_40['Vek'].replace('90 a viac rokov', '90'))
table_40 = replace_with_nan(table_40)
table_40

,Kód štátu,Názov štátu,Kód oblasti,Kód kraja,Názov oblasti,Názov kraja,Pohlavie,Vek,Vzdelanie,Súčasná ekonomická aktivita,abs.
0,SK0,Slovenská republika,SK01,SK010,Bratislavský kraj,Bratislavský kraj,muž,0,bez ukončeného vzdelania – osoby vo veku 0-14 ...,dieťa do začatia povinnej školskej dochádzky,4261
1,SK0,Slovenská republika,SK01,SK010,Bratislavský kraj,Bratislavský kraj,muž,1,bez ukončeného vzdelania – osoby vo veku 0-14 ...,dieťa do začatia povinnej školskej dochádzky,4464
2,SK0,Slovenská republika,SK01,SK010,Bratislavský kraj,Bratislavský kraj,muž,2,bez ukončeného vzdelania – osoby vo veku 0-14 ...,dieťa do začatia povinnej školskej dochádzky,4834
3,SK0,Slovenská republika,SK01,SK010,Bratislavský kraj,Bratislavský kraj,muž,3,bez ukončeného vzdelania – osoby vo veku 0-14 ...,dieťa do začatia povinnej školskej dochádzky,4736
4,SK0,Slovenská republika,SK01,SK010,Bratislavský kraj,Bratislavský kraj,muž,4,bez ukončeného vzdelania – osoby vo veku 0-14 ...,dieťa do začatia povinnej školskej dochádzky,4650
...,...,...,...,...,...,...,...,...,...,...,...
91591,SK0,Slovenská republika,SK04,SK042,Východné Slovensko,Košický kraj,žena,90,vysokoškolské vzdelanie (bližšie neuvedené),dôchodca,3
91592,SK0,Slovenská republika,SK04,SK042,Východné Slovensko,Košický kraj,žena,90,vysokoškolské vzdelanie - 1. stupeň (Bc.),dôchodca,9
91593,SK0,Slovenská republika,SK04,SK042,Východné Slovensko,Košický kraj,žena,90,vysokoškolské vzdelanie - 2. stupeň (Ing.; Mgr...,dôchodca,37
91594,SK0,Slovenská republika,SK04,SK042,Východné Slovensko,Košický kraj,žena,90,vysokoškolské vzdelanie - 3. stupeň (PhD.; a i.),dôchodca,8


In [4]:
def by_age(data, query, hue=None):
    data = data.query(query)
    if hue is None:
        aggregated = data.groupby(['Vek'])['abs.'].sum().rename('count').reset_index()
        return px.line(aggregated, x='Vek', y='count')
    else:
        aggregated = data.groupby(['Vek', hue])['abs.'].sum().rename('count').reset_index()
        return px.line(aggregated, x='Vek', y='count', color=hue)

#("Pohlavie" == "muž" or "Pohlavie" == "žena")
by_age(table_40, 'Vzdelanie == "vysokoškolské vzdelanie - 1. stupeň (Bc.)"', hue='Pohlavie')

In [5]:
properties = [
    'Vzdelanie',
    # 'Názov oblasti',
    'Názov kraja',
    'Pohlavie',
    'Súčasná ekonomická aktivita'
]
unique_data = [table_40[property].unique() for property in properties]
selected_options = [list(range(len(unique_data[i]))) for i in range(len(properties))]

In [6]:
selected_options_source = 'zeny_dovolenka.txt'

In [7]:
%%script true
print(*[' '.join(map(str, s)) for s in selected_options], sep='\n', file=open(selected_options_source, 'w'))

In [15]:
%%script true
with open(selected_options_source, 'r') as file:
    for i, line in enumerate(file.readlines()):
        selected_options[i] = list(map(int, line.split()))
display(selected_options)

In [10]:
%%script true
selected_options[0] = list(range(21))

In [16]:
# %%script true # Skip
app = Dash(__name__)

app.layout = html.Div(
    [
        html.H4('Enter title'),
        dcc.Input(id='title')
    ] +
    sum([[
        html.H4(f"Select {properties[i]}: "),
        dcc.Checklist(
            id=f'selected_{i}',
            options=[{'label': v, 'value': i} for i, v in enumerate(unique_data[i])],
            value=selected_options[i]
        )
    ] for i in range(len(properties))], [])
    + [
        html.H4('Select hue'),
        dcc.RadioItems(
            id='hue',
            options=[{ 'label': 'No', 'value': None }] + [{ 'label': v, 'value': i } for i, v, in enumerate(properties)],
            value=None,
        ),
        dcc.Graph(id='line-plot')
    ]
)

@app.callback(
    Output('line-plot', 'figure'),
    [Input('hue', 'value'), Input('title', 'value')] + [Input(f'selected_{i}', 'value') for i in range(len(properties))]
)
def update_figure(hue, title, *arg):
    selected_options[:] = list(arg)
    query = ' and '.join([
        '(' + ' or '.join([f"`{properties[i]}` == '{unique_data[i][s]}'" for s in selected]) + ')'
        for i, selected in enumerate(arg)
        if len(selected) > 0
    ])
    figure = by_age(table_40, query, properties[hue] if hue is not None else None)
    figure.update_layout(xaxis_title='Age', yaxis_title='Number of people', title=title)

    return figure

app.run_server(mode='inline', port=8052)